In [4]:
import msoffcrypto
import pandas as pd
import io
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
import locale
from utils import calculate_age, drop_columns, average_fill_empty, calculate_malnutrition, drop_cap_nutrition_rows, generate_gender_column_and_carelevel, check_missing_values, fill_missing_by_idno_and_mode
from dotenv import load_dotenv
import os

# Reading an encrypted Excel file
encrypted_file_path = "./datasets/InterRAI Pt 4 2505.xlsx"
target_folder = "./datasets/"
# Use password in .env
load_dotenv()
# Reading an encrypted Excel file
# encrypted_file_path = os.getenv("CSV_PATH")
password = os.getenv("PASSWORD")


with open(encrypted_file_path, "rb") as f:
    office_file = msoffcrypto.OfficeFile(f)
    office_file.load_key(password=password)
    # Decrypt the file
    decrypted = io.BytesIO()
    office_file.decrypt(decrypted)

    dfs = pd.read_excel(decrypted, sheet_name=None, engine="openpyxl")
basic_info_df = dfs["Staying Upright InterRAI"]
extra_info_df = dfs["Staying Upright demo2"]

In [5]:
# Preprocess the DataFrames
df = calculate_age(basic_info_df, extra_info_df)
df = generate_gender_column_and_carelevel(basic_info_df, extra_info_df)
# # Check for missing values
# columns_to_check = ['iJ1g', 'iJ1h', 'iJ1i', 'iJ12', 'iK1ab', 'iK1bb']
# check_missing_values(columns_to_check, df)
df = drop_columns(df)

    IDno        Age
0  01001  87.166667
1  01001  87.750000
2  01001  87.916667
3  01001  88.083333
4  01001  88.416667
5  01001  88.916667
6  01001  89.416667
7  01002  84.250000
8  01002  84.666667
9  01002  85.166667
   CareLevel  Gender
0        1.0     2.0
1        3.0     2.0
2        3.0     1.0
3        3.0     1.0
4        3.0     1.0
5        1.0     2.0
6        3.0     1.0
7        1.0     2.0
8        1.0     2.0
9        3.0     2.0


In [6]:
# Generate cap df
cap_df = drop_cap_nutrition_rows(df)
cap_df = fill_missing_by_idno_and_mode(cap_df)
cap_df = calculate_malnutrition(cap_df)
cap_df.to_pickle(target_folder + 'cap_data.pkl')
# df = pd.read_pickle(target_folder + 'cap_data.pkl')

In [7]:
# Generate mal df
mal_df = fill_missing_by_idno_and_mode(df)
mal_df = calculate_malnutrition(df)
mal_df = mal_df.drop(columns=["iK2a", "iK2g", "iG3", "iE2a", "iE2b", "iE2c", "iI1c", "iI1d", "CAP_Nutrition"])
mal_df.to_pickle(target_folder + 'mal_data.pkl')
# mal_df = pd.read_pickle(target_folder + 'mal_data.pkl')

In [2]:
import pickle
cap = pickle.load(open('./datasets/cap_data.pkl', 'rb'))


In [ ]:
import pickle

# 打开并加载 mal_data.pkl
target_folder = "./datasets/"
with open(target_folder + "mal_data.pkl", "rb") as f:
    mal = pickle.load(f)

# 查看数据类型
print(type(data))
# 根据内容进一步查看
print(data)


<class 'pandas.core.frame.DataFrame'>
       IDno Assessment_Date  A8  CAP_ADL  CAP_Activities  CAP_Behaviour  \
0     01001       23JAN2017   2        0               0              0   
1     01001       29AUG2017   2        0               0              0   
2     01001       24OCT2017   4        0               1              0   
3     01001       28DEC2017   4        2               1              0   
4     01001       16APR2018   2        1               1              0   
...     ...             ...  ..      ...             ...            ...   
3453  26026       14AUG2020   2        1               0              0   
3454  26026       10FEB2021   2        0               0              0   
3455  26026       15FEB2021   2        0               0              0   
3456  26026       14AUG2021   2        0               0              0   
3457  26026       06FEB2022   2        0               0              0   

      CAP_Bowel  CAP_Cardio  CAP_Cognitive  CAP_Communication

TESTING GPU

In [1]:
import torch
import torch.nn as nn
import time

def test_gpu():
    print("PyTorch version:", torch.__version__)
    print("CUDA version:", torch.version.cuda)
    print("CUDA available:", torch.cuda.is_available())

    if not torch.cuda.is_available():
        print("❌ GPU 不可用，模型将只能使用 CPU。")
        return

    device = torch.device("cuda")
    print("✅ 当前 GPU 设备：", torch.cuda.get_device_name(0))

    # 构造一个简单的 LSTM 模型
    class SimpleLSTM(nn.Module):
        def __init__(self, input_size=32, hidden_size=64):
            super(SimpleLSTM, self).__init__()
            self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
            self.fc = nn.Linear(hidden_size, 3)

        def forward(self, x, lengths):
            packed = nn.utils.rnn.pack_padded_sequence(x, lengths.cpu(), batch_first=True, enforce_sorted=False)
            packed_out, (ht, ct) = self.lstm(packed)
            out = self.fc(ht[-1])
            return out

    model = SimpleLSTM().to(device)

    # 生成模拟数据
    batch_size = 16
    seq_len = 10
    input_size = 32
    dummy_input = torch.randn(batch_size, seq_len, input_size).to(device)
    dummy_lengths = torch.tensor([seq_len]*batch_size).to(device)

    # 前向传播并计时
    start_time = time.time()
    with torch.no_grad():
        output = model(dummy_input, dummy_lengths)
    end_time = time.time()

    print("✅ 模型输出 device:", output.device)
    print("✅ 推理耗时: {:.6f} 秒".format(end_time - start_time))

    if str(output.device).startswith("cuda"):
        print("🎉 恭喜！模型已在 GPU 上成功运行。")
    else:
        print("⚠️ 模型没有在 GPU 上运行，可能有兼容性问题。")

if __name__ == "__main__":
    test_gpu()


PyTorch version: 2.5.1+cu121
CUDA version: 12.1
CUDA available: True
✅ 当前 GPU 设备： NVIDIA GeForce RTX 5070 Laptop GPU


f:\anaconda\envs\p4p\Lib\site-packages\torch\cuda\__init__.py:235: UserWarning: 
NVIDIA GeForce RTX 5070 Laptop GPU with CUDA capability sm_120 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_50 sm_60 sm_61 sm_70 sm_75 sm_80 sm_86 sm_90.
If you want to use the NVIDIA GeForce RTX 5070 Laptop GPU GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(


RuntimeError: CUDA error: no kernel image is available for execution on the device
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
